In [1]:
import os
import sys
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter

sys.path.insert(0,'../../')
import tensorflow as tf

from rcalgo.tf.utils.text_utils import *
from os import walk
import h5py

#from text_attention_rnn import *
from rcalgo.tf.tfmodels.dl_text_classification import *
from collections import Counter


In [2]:
train_dir = 'data/'

sentences = []
label = []
for idx, file in enumerate(['neg', 'pos']):  
    file_name = '{}{}.txt'.format(train_dir, file)
    with open(file_name, 'r') as f:
        content = set(f.readlines())
        for line in content:
            #sentences.append(' '.join(list(line.strip())))
            sentences.append(line.strip())
            label.append(idx)
sentences = np.array(sentences, dtype=np.str)

In [3]:
word_freq = word_count(sentences, word_seg=False)
term_id_map = build_word2idx(word_freq, freq_threshold=2)

In [4]:
maxlen = 300
seq_len = np.array([len(s) for s in sentences]) 
new_idx = np.arange(0,len(seq_len))[seq_len<maxlen]
label = np.array(label)[new_idx]
sentences = sentences[new_idx]

In [5]:
from common.train_config import *

args = parse_initial_training_args(
    [
        '--learning-rate', '0.1',
#        '--learning-rate', '0.0001',
        '--n-epochs', '10',
        '--gpu-id', '0',
        '--opt', 'sgd',
#        '--opt', 'adam',
        '--batch-size', '128',
        '--p-coef', '0.0',
        '--adam-wd','0.001',
        '--maxlen', '300',
        '--embedding-size', '128',
        '--position-type', '2,3,4,5',
        '--combine-mode', 'ADD',
        '--conv-num-filters', '128',
        #'--total-update', '1',
        '--lr-annealing'
        #'--bi-direction'
    ]
)

args.model = 'transformer'
#args.learning_rate = 0.1
args.model_version = 1
#if args.total_update is not None:
#    args.total_update = ((len(sentences)*0.8//args.batch_size) + 1) * args.n_epochs

In [6]:
graph_to_use = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
if args.model == 'fasttext':
    text_model = FastText(
        args, graph_to_use, word_dict=term_id_map)
    text_model.build_model(num_classes=2, dropout_keep_prob=1.0)
elif args.model == 'cnn':
    text_model = TextCNN(
        args, graph_to_use, word_dict=term_id_map)
    text_model.build_model(num_classes=2, max_conv_len=5,
                           num_filters=256, dropout_keep_prob=0.5)
elif args.model == 'deepcnn':
    text_model = TextDeepCNN(
        args, graph_to_use, word_dict=term_id_map)
    text_model.build_model(num_classes=2, num_filters_per_size=64,
                           filter_sizes=[5, 5, 3, 3, 3, 3],
                           pool_sizes=[2, 2, None, None, None, -1],
                           fully_layers=[256, 256], dropout_keep_prob=1.0)
elif args.model == 'transformer':
    text_model = TransformerModel(
            args, graph_to_use, word_dict=term_id_map)
    text_model.build_model(num_classes=2)
else:
    text_model = AttentionRNNModel(
        args, graph_to_use, word_dict=term_id_map)
    text_model.build_model(num_classes=2)
text_model.build_model_summary()
display(text_model.model_summary())
text_model.run([sentences, label], test_size=0.2)

W1010 14:12:46.992438 140549095376704 deprecation_wrapper.py:119] From ../tftraining/tf_object_base.py:156: The name tf.train.GradientDescentOptimizer is deprecated. Please use tf.compat.v1.train.GradientDescentOptimizer instead.

W1010 14:12:47.047628 140549095376704 deprecation.py:323] From /home/web_server/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/lookup_ops.py:1152: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1010 14:12:47.056397 140549095376704 deprecation.py:506] From /home/web_server/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:507: calling count_nonzero (from tensorflow.python.ops.math_ops) with axis is deprecated and will be removed in a future version.
Instructions for updating:
reduction_indices is deprecated, use axis instead


total parameter number: 863490


variable_name variable_shape  \
0     gpu/embedding/embedding_layer/embedding_table:0    [3139, 128]   
1            gpu/mw_conv_block/conv_1d/he_uniform/W:0  [2, 128, 128]   
2                       gpu/mw_conv_block/conv_1d/B:0          [128]   
3          gpu/mw_conv_block/conv_1d_1/he_uniform/W:0  [3, 128, 128]   
4                     gpu/mw_conv_block/conv_1d_1/B:0          [128]   
5          gpu/mw_conv_block/conv_1d_2/he_uniform/W:0  [4, 128, 128]   
6                     gpu/mw_conv_block/conv_1d_2/B:0          [128]   
7          gpu/mw_conv_block/conv_1d_3/he_uniform/W:0  [5, 128, 128]   
8                     gpu/mw_conv_block/conv_1d_3/B:0          [128]   
9                gpu/transform2_block/ln_layer/beta:0          [128]   
10              gpu/transform2_block/ln_layer/gamma:0          [128]   
11  gpu/transform2_block/multi_head_attention/conv...  [1, 128, 384]   
12  gpu/transform2_block/multi_head_attention/conv...          [384]   
13  gpu/transform2_block/multi_head_attention/conv...  [1, 128, 128]   
14  gpu/transform2_block/multi_head_attention/conv...          [128]   
15             gpu/transform2_block/ln_layer_1/beta:0          [128]   
16            gpu/transform2_block/ln_layer_1/gamma:0          [128]   
17  gpu/transform2_block/ffn_mlp/conv_1d/he_unifor...  [1, 128, 512]   
18           gpu/transform2_block/ffn_mlp/conv_1d/B:0          [512]   
19  gpu/transform2_block/ffn_mlp/conv_1d_1/he_unif...  [1, 512, 128]   
20         gpu/transform2_block/ffn_mlp/conv_1d_1/B:0          [128]   
21                    gpu/self_position_attentive/W:0     [128, 256]   
22                    gpu/self_position_attentive/b:0          [256]   
23                    gpu/self_position_attentive/u:0       [256, 1]   
24                            gpu/fully_connected/W:0       [128, 2]   
25                            gpu/fully_connected/B:0            [2]   

   parameters  
0      401792  
1       32768  
2         128  
3       49152  
4         128  
5       65536  
6         128  
7       81920  
8         128  
9         128  
10        128  
11      49152  
12        384  
13      16384  
14        128  
15        128  
16        128  
17      65536  
18        512  
19      65536  
20        128  
21      32768  
22        256  
23        256  
24        256  
25          2

total number of epochs: 10
tasks: [<tftraining.incremental_training.Task object at 0x7fd3bb130668>]
lr values: [0.1]


epoch 0 train time(sec): 3.9607231616973877


...

epoch 7 test time(sec): 0.22576498985290527
{'best_train_metric': 0.5933,
 'best_validate_metric': 0.6414,
 'epoch_index': 7,
 'train_loss': 0.6675,
 'train_metric': 0.5933,
 'validate_loss': 0.6542,
 'validate_metric': 0.6414}
lr values: [0.06666667]


epoch 8 train time(sec): 1.8896222114562988


epoch 8 test time(sec): 0.2279517650604248
{'best_train_metric': 0.5959,
 'best_validate_metric': 0.6507,
 'epoch_index': 8,
 'train_loss': 0.6619,
 'train_metric': 0.5959,
 'validate_loss': 0.6434,
 'validate_metric': 0.6507}
lr values: [0.06666667]


epoch 9 train time(sec): 1.906602382659912


epoch 9 test time(sec): 0.23382258415222168
{'best_train_metric': 0.6105,
 'best_validate_metric': 0.6682,
 'epoch_index': 9,
 'train_loss': 0.6561,
 'train_metric': 0.6105,
 'validate_loss': 0.6297,
 'validate_metric': 0.6682}
